In [ ]:
import os


try:
    # Running as a Python script (inside src/)
    this_file = os.path.abspath(__file__)
    src_root = os.path.dirname(this_file)
    project_root = os.path.dirname(src_root)

except NameError:
    # Running inside Jupyter
    cwd = os.getcwd()

    if cwd.endswith("notebooks"):
        src_root = os.path.abspath(os.path.join(cwd, ".."))
        project_root = os.path.dirname(src_root)

    elif os.path.basename(cwd) == "src":
        src_root = cwd
        project_root = os.path.dirname(src_root)

    else:
        # Running directly from project root
        project_root = cwd
        src_root = os.path.join(project_root, "src")

# ===== Canonical project paths =====
data_root    = os.path.join(project_root, "data")
prompts_root = os.path.join(project_root, "prompts")
utils_root   = os.path.join(project_root, "utils")
results_root = os.path.join(project_root, "results")
src_root     = os.path.join(project_root, "src")

print(
    f"📂 Project root : {project_root}\n"
    f"📂 Source root  : {src_root}\n"
    f"📂 Data root    : {data_root}\n"
    f"📂 Prompts root : {prompts_root}\n"
    f"📂 Utils root   : {utils_root}\n"
    f"📂 Results root : {results_root}"
)

In [ ]:
GUIDELINES_FEW_SHOT = """
==============================
ASPECT-BASED EMOTION ANNOTATION GUIDELINES (FEW-SHOT VERSION)
==============================

Your task is to assign the correct EMOTION to a given (ASPECT, POLARITY) pair based on the review text.

You must NEVER change the aspect or polarity. You ONLY choose the emotion.

--------------------------------------
ASPECT DEFINITIONS
--------------------------------------
• food — taste, freshness, texture, quality, ingredients  
• staff — waiter/waitress/host errors, friendliness, rudeness  
• service — waiting time, ordering flow, process speed, consistency  
• ambience — atmosphere, decor, noise level, vibe  
• place — physical layout, seating, space, cleanliness  
• price — cost, value for money  
• menu — options, variety, clarity  
• miscellaneous — anything that does not fit above categories  

--------------------------------------
POLARITY DEFINITIONS
--------------------------------------
positive = praise, satisfaction, admiration  
negative = criticism, frustration, disappointment, disgust  
neutral  = factual mention without emotional meaning  

--------------------------------------
EMOTION TAXONOMY
--------------------------------------
POSITIVE emotions:
• admiration — strong praise or being impressed  
• satisfaction — pleased, content, happy with outcome  
• gratitude — thankfulness directed at a person or service  

NEGATIVE emotions:
• annoyance — irritation, mild frustration  
• disappointment — unmet expectations  
• disgust — strong negative reaction to food or experience  

NEUTRAL emotions:
• no_emotion — factual mention with no emotional meaning  
• mentioned_only — aspect is referenced but emotion is not expressed  
• mixed_emotions — reviewer expresses multiple conflicting emotional signals  

--------------------------------------
RULES
--------------------------------------
1. NEVER change the aspect or polarity.  
2. Choose EXACTLY ONE emotion from the allowed list for that aspect+polarity.  
3. Do NOT invent new categories.  
4. Your output MUST be JSON.  
5. Your reasoning MUST CONTAIN EXACTLY 20 WORDS.  

--------------------------------------
FEW-SHOT EXAMPLES
--------------------------------------

### EXAMPLE 1 (negative polarity → annoyance)
Review: "The waiter rolled his eyes when I asked a question."
Aspect: staff
Polarity: negative
Correct Emotion: annoyance
Reason: "The waiter’s rude reaction clearly indicates irritation and frustration experienced by the customer during this interaction, creating a negative emotional impression."

### EXAMPLE 2 (negative polarity → disappointment)
Review: "The pasta looked amazing but tasted completely bland."
Aspect: food
Polarity: negative
Correct Emotion: disappointment
Reason: "The dish visually impressed but failed in flavor, creating unmet expectations that lead the reviewer to express genuine disappointment about the meal."

### EXAMPLE 3 (positive polarity → admiration)
Review: "The chef crafted a beautifully balanced plate that stunned me."
Aspect: food
Polarity: positive
Correct Emotion: admiration
Reason: "The reviewer expresses strong appreciation and respect for the food’s exceptional quality which clearly reflects genuine admiration for the culinary experience enjoyed."

### EXAMPLE 4 (positive polarity → satisfaction)
Review: "Our meals arrived quickly and tasted great."
Aspect: service
Polarity: positive
Correct Emotion: satisfaction
Reason: "The reviewer describes timely and pleasant service that meets expectations, expressing a comfortable sense of satisfaction with the overall dining experience."

### EXAMPLE 5 (neutral polarity → mentioned_only)
Review: "The menu lists vegetarian options."
Aspect: menu
Polarity: neutral
Correct Emotion: mentioned_only
Reason: "The reviewer simply states factual menu information without expressing any emotional stance, meaning the mention carries no emotional depth or subjective evaluation."

### EXAMPLE 6 (neutral polarity → no_emotion)
Review: "There are tables near the back of the restaurant."
Aspect: place
Polarity: neutral
Correct Emotion: no_emotion
Reason: "The description of seating location is presented factually without any emotional meaning, indicating no emotional content associated with the specific aspect described."

==============================
END OF FEW-SHOT GUIDELINES
==============================
"""

In [ ]:
GUIDELINES = """
ASPECT DEFINITIONS & CUES
-----------------------------------------
AMBIENCE (Atmosphere, Environment)
• Physical atmosphere or sensory environment: decor, lighting, music/noise, cleanliness, smell, comfort, temperature, seating, overall vibe.
• Common cues: “vibe”, “atmosphere”, “environment”, “setting”, “decor”, “layout”, “cozy”, “romantic”, “noisy”, “crowded”, “dirty”.
• Positive: lovely atmosphere, cozy.
• Negative: too loud, dirty tables.
• Neutral: dimly lit.

FOOD
• Taste, flavor, appearance, freshness, quality of dishes.
• Cues: “taste”, “flavor”, “freshness”, “undercooked”, “overcooked”, “delicious”, “gross”.
• Positive: delicious pizza.
• Negative: dry meat, awful soup.
• Neutral: factual statements about dishes.

MENU
• Menu size, variety, dietary options, availability, menu layout, confusing menus.
• Cues: “selection”, “options”, “variety”, “vegan options”, “limited”, “menu unavailable”, “confusing”.
• Positive: great variety.
• Negative: unavailable items, confusing layout.
• Neutral: normal children’s menu.

PLACE (Location)
• Location, neighborhood safety, accessibility, parking, ease of finding.
• Cues: “location”, “area”, “parking”, “hard to find”, “near station”.
• Positive: perfect location.
• Negative: unsafe area, impossible parking.
• Neutral: located next to station.

PRICE (Cost, Value for Money)
• Perception of price fairness, expensive/cheap, value for quality.
• Cues: “expensive”, “overpriced”, “cheap”, “worth it”, “value”, “rip-off”.
• Positive: good prices for portions.
• Negative: overpriced, not worth the price.
• Neutral: menu costs X.

SERVICE (Process Efficiency, Speed, Organization)
• Operational service processes: wait time, order accuracy, speed, payment, reservations. Not staff attitude.
• Cues: “slow service”, “wait time”, “efficient”, “orders messed up”.
• Positive: fast service.
• Negative: long waiting time, wrong order.
• Neutral: counter service.

STAFF (Employees, Waiters, Attitude)
• When a person is explicitly mentioned. Covers friendliness, rudeness, professionalism, helpfulness, attentiveness.
• Cues: “waiter”, “server”, “manager”, “friendly”, “rude”, “ignored”, “helpful”.
• Positive: friendly staff.
• Negative: rude waiter.
• Neutral: staff uniforms.

MISCELLANEOUS
• Overall impressions not tied to a specific aspect; general experience, overall visit, or business-level comments.
• Cues: “overall”, “experience”, “visit”, “amazing place”.
• Positive: wonderful time.
• Negative: terrible experience.
• Neutral: popular restaurant.

MENTIONED_ONLY
• Aspect is referenced but expresses no opinion, no emotion, no evaluation.
• Used only when the aspect is background info supporting another aspect.
• If ANY evaluation exists → do NOT use Mentioned_only.

MIXED EMOTIONS
• Reviewer expresses two different emotions for the same aspect category.
• Examples: “loved one dish, hated the other”, “service was helpful but frustrating”.
• Not mixed: mild variation of same polarity (“good but not great”).

-----------------------------------------
EMOTION DEFINITIONS & CUES
-----------------------------------------

ADMIRATION (Positive)
• Strong positive evaluative emotion: impressed, appreciative, respectful, elevated praise.
• Cues: “amazing”, “wow”, “impressive”, “excellent”, “stunning”, “top-notch”.
• Not admiration: weak praise (“nice”), general happiness, generic appreciation.

SATISFACTION (Positive)
• Enjoyment, comfort, pleasant surprise, happiness, relief, relaxed mood.
• Cues: “enjoyed”, “happy”, “pleasant”, “relaxing”, “surprisingly good”.
• Not satisfaction: strong admiration, gratitude, purely factual neutral.

GRATITUDE (Positive)
• Thankfulness for a specific helpful action.
• Cues: “thank you”, “grateful”, “appreciate your help”, thanking a waiter for something specific.
• Not gratitude: general appreciation of quality → admiration.

ANNOYANCE (Negative)
• Mild/moderate irritation: inconvenience, frustration, noise, crowdedness, slow service.
• Cues: “annoying”, “irritating”, “frustrating”, “bothered”, “not ideal”.
• Not annoyance: strong disappointment or disgust.

DISAPPOINTMENT (Negative)
• Unmet expectations, let-down, negative confusion.
• Cues: “disappointed”, “expected more”, “let down”, “confusing”, “sadly”, “unfortunately”.
• Not disappointment: strong anger → disgust, minor irritation → annoyance.

DISGUST (Negative)
• Strong rejection: gross food, hygiene issues, fear, unsafe environment.
• Cues: “disgusting”, “gross”, “made me sick”, “filthy”, “unsafe”, “terrifying”.
• Not disgust: mild disappointment or annoyance.

NO EMOTION (Neutral)
• Factual statements, indifference, no emotional valence.
• Cues: “okay”, “fine”, “average”, “nothing special”.
• Not neutral: weak positive “nice” → satisfaction, weak negative “not great” → disappointment.

-----------------------------------------
FINAL EMOTION SET (from PDF)
Positive: Admiration, Satisfaction, Gratitude
Negative: Disappointment, Annoyance, Disgust
Neutral: No Emotion, Mixed Emotions, Mentioned_only

-----------------------------------------
ASPECT–EMOTION PAIRS (from PDF)
ambience:
  positive: Admiration, Satisfaction
  negative: Annoyance, Disappointment
  neutral: No Emotion, Mixed Emotions, Mentioned_only

food:
  positive: Admiration, Satisfaction
  negative: Disgust, Disappointment, Annoyance
  neutral: No Emotion, Mixed Emotions, Mentioned_only

menu:
  positive: Admiration, Satisfaction
  negative: Disappointment, Annoyance
  neutral: No Emotion, Mixed Emotions, Mentioned_only

place:
  positive: Admiration, Satisfaction
  negative: Disappointment, Annoyance, Disgust
  neutral: No Emotion, Mixed Emotions, Mentioned_only

price:
  positive: Admiration, Satisfaction
  negative: Disappointment, Annoyance
  neutral: No Emotion, Mixed Emotions, Mentioned_only

service:
  positive: Admiration, Satisfaction, Gratitude
  negative: Annoyance, Disappointment
  neutral: No Emotion, Mixed Emotions, Mentioned_only

staff:
  positive: Gratitude, Admiration, Satisfaction
  negative: Disappointment, Annoyance
  neutral: No Emotion, Mixed Emotions, Mentioned_only

miscellaneous:
  positive: Admiration, Satisfaction
  negative: Annoyance, Disappointment
  neutral: No Emotion, Mixed Emotions, Mentioned_only

  """

In [ ]:
# Emotion prediction script using Gemini API
import json
import os
import requests
from dotenv import load_dotenv

# ==========================================
# API SETUP
# ==========================================
load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")

MODEL = "models/gemini-2.5-flash"
URL = f"https://generativelanguage.googleapis.com/v1beta/{MODEL}:generateContent"

HEADERS = {
    "Content-Type": "application/json",
    "X-goog-api-key": API_KEY
}

# ==========================================
# PATHS
# ==========================================
IN_PATH = os.path.join(data_root, "cleaned_300.jsonl")
EMOTION_JSON = os.path.join(data_root, "emotion.json")

OUT_DIR = os.path.join(results_root, "gemini-flash")
os.makedirs(OUT_DIR, exist_ok=True)

OUT_EMO = os.path.join(OUT_DIR, "gemini_emotion_only_cleaned_300.jsonl")
OUT_EMO_R = os.path.join(OUT_DIR, "gemini_emotion_only_reasons_cleaned_300.jsonl")

# ==========================================
# LOAD FULL EMOTION TAXONOMY
# ==========================================
EMOTIONS = json.load(open(EMOTION_JSON, "r", encoding="utf-8"))
POLARITIES = ["positive", "negative", "neutral"]

# ==========================================
# PARSE GOLD INPUT (BUT IGNORE GOLD EMOTIONS)
# ==========================================
raw_data = [
    json.loads(line)
    for line in open(IN_PATH, "r", encoding="utf-8")
]

# Each row["output"] has aspect/polarity/emotion
# but EMOTION is IGNORED → the model predicts new ones


# ==========================================
# GEMINI REQUEST WRAPPER
# ==========================================
def ask_gemini(prompt):
    payload = {"contents": [{"parts": [{"text": prompt}]}]}
    r = requests.post(URL, headers=HEADERS, json=payload)
    r.raise_for_status()
    return r.json()["candidates"][0]["content"]["parts"][0]["text"].strip()


# ==========================================
# SAFE JSON PARSER
# ==========================================
def safe_json_parse(txt):
    try:
        return json.loads(txt)
    except:
        pass

    cleaned = txt.replace("```json", "").replace("```", "").strip()
    try:
        return json.loads(cleaned)
    except:
        pass

    cleaned = cleaned.replace(",]", "]").replace(",}", "}")
    try:
        return json.loads(cleaned)
    except:
        return None


# ==========================================
# EMOTION-ONLY PROMPT BUILDER
# ==========================================
def build_emotion_only_prompt(review, aspect, polarity):
    allowed = EMOTIONS[aspect][polarity]

    return f"""
You are performing EMOTION-ONLY annotation following strict official guidelines.

Below are the complete annotation guidelines that you MUST follow exactly:
{GUIDELINES}

### TASK:
You MUST NOT modify aspect or polarity.
Your ONLY task is to choose the correct EMOTION.

### Review:
\"""{review}\"""

### Aspect (DO NOT CHANGE):
{aspect}

### Polarity (DO NOT CHANGE):
{polarity}

### Allowed Emotion Categories:
{allowed}

### STRICT JSON OUTPUT:
{{
  "emotion": "...",
  "reason": "A single sentence of exactly 20 words explaining your reasoning."
}}

### RULES:
- JSON ONLY.
- Choose exactly ONE emotion from allowed list.
- Do NOT invent categories.
- Do NOT output anything except JSON.
- "reason" MUST contain exactly 20 words.

Return ONLY JSON.
"""


# ==========================================
# CALL GEMINI FOR ONE EMOTION
# ==========================================
def annotate_emotion_only(review, aspect, polarity):
    prompt = build_emotion_only_prompt(review, aspect, polarity)

    parsed = None
    for _ in range(3):
        response = ask_gemini(prompt)
        parsed = safe_json_parse(response)

        if isinstance(parsed, dict) and "emotion" in parsed:
            break

    if not isinstance(parsed, dict):
        print("JSON ERROR →", response)
        return None, "Reason unavailable"

    emo = parsed.get("emotion", "").strip()
    reason = parsed.get("reason", "").strip()

    # Capitalize for consistency
    if emo:
        emo = emo[0].upper() + emo[1:]

    # Validate → fallback to first allowed if invalid
    allowed = EMOTIONS[aspect][polarity]
    if emo not in allowed:
        emo = allowed[0]

    return emo, reason


# ==========================================
# RUN EMOTION-ONLY ANNOTATION
# ==========================================
emotion_only_results = []
emotion_only_reasons = []

for row in raw_data:
    review = row["input"]
    gold = row["output"]  # ignore emotion; use aspect/polarity only

    annotated = []
    reasons = []

    for t in gold:
        asp = t["aspect"]
        pol = t["polarity"]

        emo, rtext = annotate_emotion_only(review, asp, pol)

        annotated.append({
            "aspect": asp,
            "polarity": pol,
            "emotion": emo
        })

        reasons.append({
            "aspect": asp,
            "polarity": pol,
            "emotion": emo,
            "reason": rtext
        })

    emotion_only_results.append({
        "input": review,
        "output": annotated
    })

    emotion_only_reasons.append({
        "input": review,
        "details": reasons
    })


# ==========================================
# SAVE OUTPUT FILES
# ==========================================
with open(OUT_EMO, "w", encoding="utf-8") as f:
    for r in emotion_only_results:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

with open(OUT_EMO_R, "w", encoding="utf-8") as f:
    for r in emotion_only_reasons:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print("DONE →", OUT_EMO)
print("REASONS →", OUT_EMO_R)

In [ ]:
# # Full absa with aspect, polarity, emotion prediction


# import json
# import os
# import requests
# from dotenv import load_dotenv

# # -----------------------------
# # API setup
# # -----------------------------
# load_dotenv()
# API_KEY = os.getenv("GEMINI_API_KEY")

# MODEL = "models/gemini-2.5-flash"
# URL = f"https://generativelanguage.googleapis.com/v1beta/{MODEL}:generateContent"

# HEADERS = {
#     "Content-Type": "application/json",
#     "X-goog-api-key": API_KEY
# }

# # -----------------------------
# # Paths
# # -----------------------------
# IN_PATH = os.path.join(data_root, "daniel_50.jsonl")
# EMOTION_JSON = os.path.join(data_root, "emotion.json")

# OUT_DIR = os.path.join(results_root, "gemini-flash")
# os.makedirs(OUT_DIR, exist_ok=True)

# OUT_ANNOT_PATH = os.path.join(OUT_DIR, "gemini_annotated_aspect_polarity_daniel_50.jsonl")
# OUT_REASON_PATH = os.path.join(OUT_DIR, "gemini_reasons_daniel_50.jsonl")

# # -----------------------------
# # Load emotion taxonomy
# # -----------------------------
# EMOTIONS = json.load(open(EMOTION_JSON, "r", encoding="utf-8"))

# ASPECTS = list(EMOTIONS.keys())
# POLARITIES = ["positive", "negative", "neutral"]

# allowed_lookup = {
#     (aspect, polarity): EMOTIONS[aspect][polarity]
#     for aspect in EMOTIONS
#     for polarity in EMOTIONS[aspect]
# }

# # ----------------------------------------------------
# # Gemini request
# # ----------------------------------------------------
# def ask_gemini(prompt):
#     payload = {"contents": [{"parts": [{"text": prompt}]}]}
#     r = requests.post(URL, headers=HEADERS, json=payload)
#     r.raise_for_status()
#     return r.json()["candidates"][0]["content"]["parts"][0]["text"].strip()

# # ----------------------------------------------------
# # JSON repair
# # ----------------------------------------------------
# def safe_json_parse(txt):
#     try:
#         return json.loads(txt)
#     except:
#         pass

#     cleaned = txt.replace("```json", "").replace("```", "").strip()
#     try:
#         return json.loads(cleaned)
#     except:
#         pass

#     cleaned = cleaned.replace(",]", "]").replace(",}", "}")
#     try:
#         return json.loads(cleaned)
#     except:
#         return None

# # ----------------------------------------------------
# # Build prompt (uses GUIDELINES from previous cell)
# # ----------------------------------------------------
# def build_prompt(review):
#     return f"""
# You are performing ABSA (Aspect-Based Sentiment & Emotion) annotation.

# Follow these official annotation guidelines:
# {GUIDELINES}

# ### Allowed aspects:
# {ASPECTS}

# ### Allowed polarities:
# {POLARITIES}

# ### Allowed emotions:
# {json.dumps(EMOTIONS, indent=2)}

# ### STRICT OUTPUT FORMAT:
# {{
#   "triples": [
#     {{"aspect": "...", "polarity": "...", "emotion": "..."}}
#   ],
#   "reason": "A single sentence of exactly 20 words explaining your reasoning."
# }}

# ### RULES:
# - Output MUST be ONLY valid JSON.
# - No markdown, no natural-language explanation outside JSON.
# - Emotion must belong to allowed list for the given (aspect, polarity).
# - If no aspects appear → return `"triples": []` plus a 20-word reason.
# - `reason` MUST contain exactly 20 words.

# ### Review:
# \"""{review}\"""

# Return ONLY the JSON.
# """

# # ----------------------------------------------------
# # Annotation logic
# # ----------------------------------------------------
# def annotate_full(review):
#     prompt = build_prompt(review)

#     parsed = None
#     for _ in range(3):
#         response = ask_gemini(prompt)
#         parsed = safe_json_parse(response)
#         if isinstance(parsed, dict) and "triples" in parsed:
#             break

#     if not isinstance(parsed, dict):
#         print("JSON ERROR →", response)
#         return [], "Reason unavailable"

#     triples = parsed.get("triples", [])
#     reason = parsed.get("reason", "").strip()

#     # Validate triples
#     final = []
#     for item in triples:
#         asp = item.get("aspect")
#         pol = item.get("polarity")
#         emo = item.get("emotion", "")

#         if (asp, pol) not in allowed_lookup:
#             continue

#         allowed = allowed_lookup[(asp, pol)]

#         # Normalize
#         emo = emo.strip()
#         if emo:
#             emo = emo[0].upper() + emo[1:]

#         if emo not in allowed:
#             emo = allowed[0]

#         final.append({
#             "aspect": asp,
#             "polarity": pol,
#             "emotion": emo
#         })

#     return final, reason

# # ----------------------------------------------------
# # Load input
# # ----------------------------------------------------
# raw_data = [
#     json.loads(line)
#     for line in open(IN_PATH, "r", encoding="utf-8")
# ]

# # ----------------------------------------------------
# # Annotate all
# # ----------------------------------------------------
# results = []
# reasons = []

# for row in raw_data:
#     review = row["input"]
#     triples, reason = annotate_full(review)

#     results.append({
#         "input": review,
#         "output": triples
#     })

#     reasons.append({
#         "input": review,
#         "triples": triples,
#         "reason": reason
#     })

# # ----------------------------------------------------
# # Save Files
# # ----------------------------------------------------
# with open(OUT_ANNOT_PATH, "w", encoding="utf-8") as f:
#     for r in results:
#         f.write(json.dumps(r, ensure_ascii=False) + "\n")

# with open(OUT_REASON_PATH, "w", encoding="utf-8") as f:
#     for r in reasons:
#         f.write(json.dumps(r, ensure_ascii=False) + "\n")

# print("DONE →", OUT_ANNOT_PATH)
# print("REASONS →", OUT_REASON_PATH)


